In [1]:
from datasets import load_dataset, load_dataset_builder  # pip install datasets

c:\Users\LUKE_SARGEN\anaconda3\envs\pytorch110cu113\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\LUKE_SARGEN\anaconda3\envs\pytorch110cu113\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\Users\LUKE_SARGEN\anaconda3\envs\pytorch110cu113\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
dataset_name = "wikitext"
dataset_config = "wikitext-103-raw-v1"

In [3]:
dataset = load_dataset(dataset_name, dataset_config)

Reusing dataset wikitext (C:\Users\LUKE_SARGEN\.cache\huggingface\datasets\wikitext\wikitext-103-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
100%|██████████| 3/3 [00:00<00:00, 12.92it/s]


In [4]:
dataset["test"].__dict__.keys()

dict_keys(['_info', '_split', '_indexes', '_data', '_indices', '_format_type', '_format_kwargs', '_format_columns', '_output_all_columns', '_fingerprint'])

In [5]:
dataset["test"].features

{'text': Value(dtype='string', id=None)}

In [4]:
splits = ['train', 'validation', 'test']
filenames = [f"{dataset_config}-{split}.txt" for split in splits]
for split, filename in zip(splits, filenames):
    print(len(dataset[split]))
    with open(filename, "w+", encoding="UTF-8") as f:
        f.writelines(dataset[split]["text"])

1801350
3760
4358


In [26]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

In [27]:
test_str = "Tripled up to 99,999, and that's only the first round."

In [28]:
tokenizer = Tokenizer(models.BPE())
pre_tokenizer = pre_tokenizers.Sequence([
    pre_tokenizers.Digits(individual_digits=True),
    pre_tokenizers.ByteLevel(add_prefix_space=False)
])
tokenizer.pre_tokenizer = pre_tokenizer

In [29]:
trainer = trainers.BpeTrainer(vocab_size=2048, special_tokens=["<|endoftext|>"])
tokenizer.train(files=filenames[1:], trainer=trainer)

In [30]:
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
tokenizer.decoder = decoders.ByteLevel()

In [31]:
from transformers import PreTrainedTokenizerFast
wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    model_max_length=512,
    bos_token="<|endoftext|>",
    eos_token="<|endoftext|>",
)
wrapped_tokenizer

PreTrainedTokenizerFast(name_or_path='', vocab_size=2048, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>'})

In [32]:
wrapped_tokenizer.save_pretrained("tokenizer-fast")

('tokenizer-fast\\tokenizer_config.json',
 'tokenizer-fast\\special_tokens_map.json',
 'tokenizer-fast\\tokenizer.json')

In [33]:
loaded_tokenizer = PreTrainedTokenizerFast.from_pretrained("tokenizer-fast")

In [39]:
output = tokenizer.encode("Hello, y'all! How are you 😁 ?")
print(output.tokens)
print(output.ids)

['H', 'ell', 'o', ',', 'Ġy', "'", 'all', '!', 'ĠH', 'ow', 'Ġare', 'Ġy', 'ou', 'Ġ', 'Ł', 'ĺ', 'ģ', 'Ġ', '?']
[40, 426, 76, 12, 425, 7, 352, 1, 270, 273, 396, 425, 217, 148, 180, 173, 150, 148, 31]


In [36]:
output = loaded_tokenizer.encode("Hello, y'all! How are you 😁 ?")
print(output)
# decode = tokenizer.batch_decode(output)
# print(decode)

[40, 426, 76, 12, 425, 7, 352, 1, 270, 273, 396, 425, 217, 148, 180, 173, 150, 148, 31]


In [2]:
from transformers import GPT2TokenizerFast
gpt_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

c:\Users\LUKE_SARGEN\anaconda3\envs\pytorch110cu113\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\LUKE_SARGEN\anaconda3\envs\pytorch110cu113\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\Users\LUKE_SARGEN\anaconda3\envs\pytorch110cu113\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
gpt_tokenizer("Hello world")

{'input_ids': [15496, 995], 'attention_mask': [1, 1]}

In [4]:
gpt_tokenizer(" Hello world")

{'input_ids': [18435, 995], 'attention_mask': [1, 1]}